In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("lab1").setMaster('local')
sc = SparkContext.getOrCreate(conf=conf)

In [2]:
warandpeace = sc.textFile("warandsociety.txt")
warandpeace.count()

12851

In [4]:
nilFile = sc.textFile("nil")
#nilFile.count()

In [5]:
warandpeace.take(10)

['Лев Николаевич Толстой',
 'Война и мир. Книга 1',
 '',
 'Война и мир – 1',
 '',
 ' ',
 ' http://www.lib.ru',
 '',
 'Аннотация ',
 '']

In [6]:
warandpeace.getNumPartitions

<bound method RDD.getNumPartitions of warandsociety.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0>

Обработка текста

In [7]:
linesWithWar = warandpeace.filter(lambda x: "война" in x)

In [8]:
linesWithWar.first()

"– Еh bien, mon prince. Genes et Lucques ne sont plus que des apanages, des поместья, de la famille Buonaparte. Non, je vous previens, que si vous ne me dites pas, que nous avons la guerre, si vous vous permettez encore de pallier toutes les infamies, toutes les atrocites de cet Antichrist (ma parole, j'y crois) – je ne vous connais plus, vous n'etes plus mon ami, vous n'etes plus мой верный раб, comme vous dites. [Ну, что, князь, Генуа и Лукка стали не больше, как поместьями фамилии Бонапарте. Нет, я вас предупреждаю, если вы мне не скажете, что у нас война, если вы еще позволите себе защищать все гадости, все ужасы этого Антихриста (право, я верю, что он Антихрист) – я вас больше не знаю, вы уж не друг мой, вы уж не мой верный раб, как вы говорите.] Ну, здравствуйте, здравствуйте. Je vois que je vous fais peur, [Я вижу, что я вас пугаю,] садитесь и рассказывайте."

In [9]:
def time(f):    
    import time
    t = time.process_time()
    f()
    print(f"Elapsed time: {int((time.process_time() - t)*1e9)} ns")

In [10]:
linesWithWar.cache()

PythonRDD[10] at RDD at PythonRDD.scala:53

In [11]:
time(lambda: linesWithWar.count() )
time(lambda: linesWithWar.count() )

Elapsed time: 11982444 ns
Elapsed time: 8917253 ns


In [50]:
wordCounts = linesWithWar.flatMap(lambda line: line.split(" ")).map(lambda word: re.sub('[A-ZА-Я]', lambda m: m.group(0).lower(), word)).map(lambda word: re.sub(r'[^\w\s]','',word)).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

In [54]:
#wordCounts.saveAsTextFile("warandpeace_histogram7.txt")

In [53]:
 #!hadoop fs -cat warandpeace_histogram7.txt/*

In [20]:
# Упражнение (регулярные выражения)
wordCounts = linesWithWar.flatMap(lambda line: line.split(" ")).map(lambda word: re.sub('[A-ZА-Я]', lambda m: m.group(0).lower(), word)).map(lambda word: re.sub(r'[^\w\s]','',word)).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
#wordCounts.saveAsTextFile("warandpeace_histogramRegular.txt")

Операции с множествами

In [21]:
a = sc.parallelize([1,2,3,4])
b = sc.parallelize([3,4,6,7])

In [22]:
a.union(b).distinct().collect()

[2, 4, 6, 1, 3, 7]

In [23]:
a.intersection(b).collect()

[4, 3]

In [24]:
a.subtract(b).collect()

[2, 1]

Общие переменные

In [25]:
broadcastVar = sc.broadcast([1,2,3])

In [26]:
broadcastVar.value

[1, 2, 3]

In [27]:
accum = sc.accumulator(0)

In [28]:
sc.parallelize([1,2,3,4]).foreach(lambda x: accum.add(x))

In [29]:
accum.value

10

In [30]:
pair = ('a', 'b')

In [31]:
pair[0]

'a'

In [32]:
pair[1]

'b'

Топ-10 популярных номеров такси

In [33]:
taxi = sc.textFile("nyctaxi.csv")

In [34]:
for t in taxi.take(5):
    print(t)

"_id","_rev","dropoff_datetime","dropoff_latitude","dropoff_longitude","hack_license","medallion","passenger_count","pickup_datetime","pickup_latitude","pickup_longitude","rate_code","store_and_fwd_flag","trip_distance","trip_time_in_secs","vendor_id"
"29b3f4a30dea6688d4c289c9672cb996","1-ddfdec8050c7ef4dc694eeeda6c4625e","2013-01-11 22:03:00",+4.07033460000000E+001,-7.40144200000000E+001,"A93D1F7F8998FFB75EEF477EB6077516","68BC16A99E915E44ADA7E639B4DD5F59",2,"2013-01-11 21:48:00",+4.06760670000000E+001,-7.39810790000000E+001,1,,+4.08000000000000E+000,900,"VTS"
"2a80cfaa425dcec0861e02ae44354500","1-b72234b58a7b0018a1ec5d2ea0797e32","2013-01-11 04:28:00",+4.08190960000000E+001,-7.39467470000000E+001,"64CE1B03FDE343BB8DFB512123A525A4","60150AA39B2F654ED6F0C3AF8174A48A",1,"2013-01-11 04:07:00",+4.07280540000000E+001,-7.40020370000000E+001,1,,+8.53000000000000E+000,1260,"VTS"
"29b3f4a30dea6688d4c289c96758d87e","1-387ec30eac5abda89d2abefdf947b2c1","2013-01-11 22:02:00",+4.07277180000000E+00

In [35]:
import itertools
taxi.mapPartitionsWithIndex(lambda idx, it:  itertools.islice(it,1,None) if (idx==0) else it  )

PythonRDD[70] at RDD at PythonRDD.scala:53

In [36]:
taxiParse = taxi.map(lambda line: line.split(","))

In [37]:
taxiMedKey = taxiParse.map(lambda row: (row[6], 1))

In [38]:
taxiMedCounts = taxiMedKey.reduceByKey(lambda v1, v2: v1+v2)

In [39]:
top10 = taxiMedCounts.map(lambda x: x[::-1]).top(10)
for x in top10:
    print(x[::-1])

('"AB44AD9A03B7CFAF3925103BDCC0AF23"', 44)
('"71CACFBADF9568AAE88A843DB511D172"', 41)
('"6483B9BFCB216EC88986EA3AB13064E7"', 41)
('"4C73459B430339981D78795300433438"', 41)
('"67E71D24AF704D814A0A825005ADA72E"', 40)
('"02E5A4136FD0A775A023A005A4EABC62"', 40)
('"9DFBCD218E7116F34C044F0680A0FB8A"', 39)
('"8DEB70907D00AA1D7FF5E2683240549B"', 39)
('"7989C2AB3F345F4AB54D3CF1E0480D67"', 39)
('"6C9F67DF658DC5636F9E7752F203F70A"', 39)


In [40]:
taxiCounts = taxi.map(lambda line: line.split(",")).map(lambda row: (row[6],1)).reduceByKey(lambda a,b: a + b)

In [41]:
taxiCounts.cache()

PythonRDD[80] at RDD at PythonRDD.scala:53

In [42]:
time(lambda: taxiCounts.count())
time(lambda: taxiCounts.count())

Elapsed time: 13821253 ns
Elapsed time: 7931078 ns


In [43]:
import pyspark
taxi.persist(storageLevel=pyspark.StorageLevel.MEMORY_ONLY)

nyctaxi.csv MapPartitionsRDD[68] at textFile at NativeMethodAccessorImpl.java:0

In [49]:
!spark-submit --master yarn --deploy-mode cluster L1_noninteractive_bike_analysis_python.py --conf spark.yarn.submit.waitAppCompletion=true